# Entrenamiento XGBOOST

In [1]:
! pip install xgboost
! pip install scikit-learn
! pip install pandas
! pip install numpy 
! pip install matplotlib


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import xgboost as xgb
print("XGBoost version:", xgb.__version__)

XGBoost version: 3.1.2


## Division de datos

In [7]:
import pandas as pd
import numpy as np
import os

file_dict = {
    25037: {'Clasificacion': 'Seco', 'ruta_origen': '../data/data_conagua_clasificada/Seco/NR_25037.csv', 'ruta_salida': './XGBOOST_dataset_splits/Seco/NR_25037/'},
    25033: {'Clasificacion': 'Templado', 'ruta_origen': '../data/data_conagua_clasificada/Templado/NR_25033.csv', 'ruta_salida': './XGBOOST_dataset_splits/Templado/NR_25033/'},
    25046: {'Clasificacion': 'Humedo', 'ruta_origen': '../data/data_conagua_clasificada/Tropical/NR_25046.csv', 'ruta_salida': './XGBOOST_dataset_splits/Tropical/NR_25046/'}
}

TRAIN_RATIO = 0.70
TEST_RATIO = 0.20

for file_id, file_info in file_dict.items():
    ruta_origen = file_info['ruta_origen']
    ruta_salida_base = file_info['ruta_salida']
    
    
    df = pd.read_csv(ruta_origen, index_col=0, parse_dates=True).asfreq('D')
    df = df.sort_index()
    df = df.asfreq('D')
    
    total_rows = len(df)
    
    train_end = int(total_rows * TRAIN_RATIO)
    test_end = int(total_rows * (TRAIN_RATIO + TEST_RATIO))
    
    df_train = df.iloc[:train_end]
    df_test = df.iloc[train_end:test_end]
    df_val = df.iloc[test_end:]
    
    splits = {'train':{'df': df_train, 'ruta_salida': os.path.join(ruta_salida_base, 'train')},
              'test': {'df': df_test, 'ruta_salida': os.path.join(ruta_salida_base, 'test')},
              'val':  {'df': df_val, 'ruta_salida': os.path.join    (ruta_salida_base, 'val')}}
    
    for split_name, split_info in splits.items():
        save_path = split_info['ruta_salida']
        os.makedirs(save_path, exist_ok=True)
        
        print(f"Guardando {split_name} data en {save_path}")
        split_info['df'].to_csv(os.path.join(save_path, f"{split_name}_data.csv"))
    
    

Guardando train data en ./XGBOOST_dataset_splits/Seco/NR_25037/train
Guardando test data en ./XGBOOST_dataset_splits/Seco/NR_25037/test
Guardando val data en ./XGBOOST_dataset_splits/Seco/NR_25037/val
Guardando train data en ./XGBOOST_dataset_splits/Templado/NR_25033/train
Guardando test data en ./XGBOOST_dataset_splits/Templado/NR_25033/test
Guardando val data en ./XGBOOST_dataset_splits/Templado/NR_25033/val
Guardando train data en ./XGBOOST_dataset_splits/Tropical/NR_25046/train
Guardando test data en ./XGBOOST_dataset_splits/Tropical/NR_25046/test
Guardando val data en ./XGBOOST_dataset_splits/Tropical/NR_25046/val


## Entrenamiento

In [10]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from xgboost import XGBRegressor, XGBClassifier
import joblib

directory = './XGBOOST_dataset_splits/'
model_output_dir = './XGBOOST_trained_models/'


#Hiperparametros
params = {
    'estimator': XGBRegressor(
    n_estimators= 100, #Numero de arboles
    learning_rate= 0.05, #Tasa de aprendizaje
    max_depth= 5, #Profundidad maxima de los arboles, baja para evitar sobreajuste
    n_jobs= -1, #Usar todos los nucleos disponibles
    random_state= 42, #Para reproducibilidad
    tree_method= 'hist' #Metodo de construccion de arboles eficiente
    ),
    'max_iter': 10, #Numero maximo de iteraciones para el imputador
    'random_state': 42 #Para reproducibilidad
}

columnas_objetivo = ['PRECIP', 'EVAP', 'TMAX', 'TMIN']

estaciones = {
    'NR_25037': 'Seco',
    'NR_25033': 'Templado',
    'NR_25046': 'Tropical'
}

for region, estacion in estaciones.items():
    archivo_train = os.path.join(directory, estacion, region, 'train', 'train_data.csv')
    
    print(f"Cargando datos de entrenamiento desde {archivo_train}")
    
    df_train = pd.read_csv(archivo_train, index_col=0, parse_dates=True).asfreq('D')
    
    df = df_train.copy()
    df['dayofyear'] = df.index.dayofyear #Feature ciclico para el día del año
    df['month'] = df.index.month  #Feature ciclico para el mes
    df['sin_dayofyear'] = np.sin(2 * np.pi * df['dayofyear'] / 365.25) #Feature ciclico seno (Necesario para capturar patrones estacionales por distancia entre fecha)
    df['cos_dayofyear'] = np.cos(2 * np.pi * df['dayofyear'] / 365.25) #Feature ciclico coseno
    
    columnas_imputar = columnas_objetivo + ['dayofyear', 'month', 'sin_dayofyear', 'cos_dayofyear']

    imputer = IterativeImputer(**params)
    imputer.fit(df[columnas_imputar])
    
    modelo_imputacion_path = os.path.join(model_output_dir, estacion, region)
    os.makedirs(modelo_imputacion_path, exist_ok=True)
    joblib.dump(imputer, os.path.join(modelo_imputacion_path, 'imputer_model.joblib'))
    print(f"Modelo de imputacipn guardado en {modelo_imputacion_path}")
    


Cargando datos de entrenamiento desde ./XGBOOST_dataset_splits/Seco\NR_25037\train\train_data.csv


c:\Users\Sarah\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Modelo de imputacipn guardado en ./XGBOOST_trained_models/Seco\NR_25037
Cargando datos de entrenamiento desde ./XGBOOST_dataset_splits/Templado\NR_25033\train\train_data.csv


c:\Users\Sarah\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Modelo de imputacipn guardado en ./XGBOOST_trained_models/Templado\NR_25033
Cargando datos de entrenamiento desde ./XGBOOST_dataset_splits/Tropical\NR_25046\train\train_data.csv


c:\Users\Sarah\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\impute\_iterative.py:895: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


Modelo de imputacipn guardado en ./XGBOOST_trained_models/Tropical\NR_25046


## Testing de imputador

In [22]:
import pandas as pd
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

directory = {
    25037:{
        'Clasificacion': 'Seco',
        'ruta_input_test': './XGBOOST_dataset_splits/Seco/NR_25037/test/test_data.csv',
        'ruta_salida': './data_analysis/Performance_metrics/Seco/NR_25037/test/',
        'ruta_modelo_imputacion': './XGBOOST_trained_models/Seco/NR_25037/imputer_model.joblib'
    },
    25033:{
        'Clasificacion': 'Templado',
        'ruta_input_test': './XGBOOST_dataset_splits/Templado/NR_25033/test/test_data.csv',
        'ruta_salida': './data_analysis/Performance_metrics/Templado/NR_25033/test/',
        'ruta_modelo_imputacion': './XGBOOST_trained_models/Templado/NR_25033/imputer_model.joblib'
    },
    25046:{
        'Clasificacion': 'Tropical',
        'ruta_input_test': './XGBOOST_dataset_splits/Tropical/NR_25046/test/test_data.csv',
        'ruta_salida': './data_analysis/Performance_metrics/Tropical/NR_25046/test/',
        'ruta_modelo_imputacion': './XGBOOST_trained_models/Tropical/NR_25046/imputer_model.joblib'
    },
}

columnas_objetivo = ['PRECIP', 'EVAP', 'TMAX', 'TMIN']

# Horizontes
horizontes = [0, 3, 7, 31, 180, 365, 730, float('inf')]
etiquetas_horizontes = ['1d', '3d', '7d', '31d', '180d', '365d', '730d', 'full']

print("--- INICIANDO VALIDACIÓN DE IMPUTACIÓN ---")

for estacion, info in directory.items():
    test_path = info['ruta_input_test']
    modelo_imputacion_path = info['ruta_modelo_imputacion']
    ruta_salida = info['ruta_salida']
    os.makedirs(ruta_salida, exist_ok=True)
    
    print(f"\nProcesando Estación: {estacion}...")

    if not os.path.exists(test_path):
        print(f"   [SKIP] No existe archivo test: {test_path}")
        continue
        
    df_test_raw = pd.read_csv(test_path, index_col=0, parse_dates=True).asfreq('D')
    
    try:
        imputer = joblib.load(modelo_imputacion_path)
    except Exception as e:
        print(f"   [ERROR] No se pudo cargar el imputer: {e}")
        continue
    

    df_para_imputar = df_test_raw.copy()
    df_para_imputar['dayofyear'] = df_para_imputar.index.dayofyear
    df_para_imputar['month'] = df_para_imputar.index.month
    df_para_imputar['sin_dayofyear'] = np.sin(2 * np.pi * df_para_imputar.index.dayofyear / 365.25)
    df_para_imputar['cos_dayofyear'] = np.cos(2 * np.pi * df_para_imputar.index.dayofyear / 365.25)
    
    columnas_mice = columnas_objetivo + ['dayofyear', 'month', 'sin_dayofyear', 'cos_dayofyear']
    
    for objetivo in columnas_objetivo:
        if objetivo not in df_test_raw.columns:
            continue
            
        print(f"   Evaluando variable: {objetivo}")
            
        df_mascara = df_para_imputar.copy()
        indices_validos = df_mascara[df_mascara[objetivo].notna()].index
        
        if len(indices_validos) == 0:
            print(f"      No hay datos válidos para evaluar {objetivo}. Saltando...")
            continue
        
        #Guardar Realidad y Borrar Datos
        datos_reales = df_mascara.loc[indices_validos, objetivo].copy()
        df_mascara.loc[indices_validos, objetivo] = np.nan # Borramos para obligar a imputar
        
        try:
            matriz_imputada = imputer.transform(df_mascara[columnas_mice])
            df_imputada = pd.DataFrame(matriz_imputada, columns=columnas_mice, index=df_mascara.index)

            y_pred_imputado = df_imputada.loc[indices_validos, objetivo]
            
            df_resultado = pd.DataFrame({
                'Real': datos_reales,
                'Imputado': y_pred_imputado
            })
            
            # Calcular Horizontes
            fecha_inicio = df_resultado.index.min()
            df_resultado['Día'] = (df_resultado.index - fecha_inicio).days + 1
            df_resultado['rango'] = pd.cut(df_resultado['Día'], bins=[-1] + horizontes, labels=etiquetas_horizontes)
            
            lista_metricas = []
            
            for etiqueta in etiquetas_horizontes:
                subset = df_resultado[df_resultado['rango'] == etiqueta]
                count = len(subset)
                
                if count > 0:
                    y_true = subset['Real']
                    y_pred = subset['Imputado']
                    
                    mse = np.mean((y_true - y_pred)**2)
                    rmse = np.sqrt(mse)
                    mae = np.mean(np.abs(y_true - y_pred))
                    
                    # MAPE seguro
                    mascara_no_cero = y_true != 0
                    if np.sum(mascara_no_cero) > 0:
                        mape = np.mean(np.abs((y_true[mascara_no_cero] - y_pred[mascara_no_cero]) / y_true[mascara_no_cero]))
                    else:
                        mape = 0.0
                else:
                    mse = 0.0
                    rmse = 0.0
                    mae = 0.0
                    mape = 0.0
                
                lista_metricas.append({
                    'Horizonte': etiqueta,
                    'MSE': round(mse, 4),
                    'RMSE': round(rmse, 4),
                    'MAE': round(mae, 4),
                    'MAPE': round(mape, 4),
                    'Count': count
                })
                
            
            df_metricas = pd.DataFrame(lista_metricas)
            archivo_csv_metrics = os.path.join(ruta_salida, f'/{objetivo}/metrics_test_{objetivo}.csv')
            os.makedirs(os.path.dirname(archivo_csv_metrics), exist_ok=True)
            df_metricas.to_csv(archivo_csv_metrics, index=False)
            
            # Guardar Imagen Tabla
            fig, ax = plt.subplots(figsize=(10, 4))
            ax.axis('off')
            tabla = ax.table(cellText=df_metricas.values,
                             colLabels=df_metricas.columns,
                             cellLoc='center',
                             loc='center')
            tabla.scale(1, 1.5)
            tabla.auto_set_font_size(False)
            tabla.set_fontsize(9)
            
            plt.title(f'Validación Imputación: {objetivo} - {estacion}')
            plt.savefig(os.path.join(ruta_salida, f'metrics_table_{objetivo}.png'), bbox_inches='tight')
            plt.close()
            
            print(f"      [OK] Guardado: {archivo_csv_metrics}")
            
        except Exception as e:
            print(f"      [ERROR] Falló imputación de {objetivo}: {e}")


--- INICIANDO VALIDACIÓN DE IMPUTACIÓN ---

Procesando Estación: 25037...
   Evaluando variable: PRECIP
      [OK] Guardado: /PRECIP/metrics_test_PRECIP.csv
   Evaluando variable: EVAP
      [OK] Guardado: /EVAP/metrics_test_EVAP.csv
   Evaluando variable: TMAX
      [OK] Guardado: /TMAX/metrics_test_TMAX.csv
   Evaluando variable: TMIN
      [OK] Guardado: /TMIN/metrics_test_TMIN.csv

Procesando Estación: 25033...
   Evaluando variable: PRECIP
      [OK] Guardado: /PRECIP/metrics_test_PRECIP.csv
   Evaluando variable: EVAP
      [OK] Guardado: /EVAP/metrics_test_EVAP.csv
   Evaluando variable: TMAX
      [OK] Guardado: /TMAX/metrics_test_TMAX.csv
   Evaluando variable: TMIN
      [OK] Guardado: /TMIN/metrics_test_TMIN.csv

Procesando Estación: 25046...
   Evaluando variable: PRECIP
      [OK] Guardado: /PRECIP/metrics_test_PRECIP.csv
   Evaluando variable: EVAP
      [OK] Guardado: /EVAP/metrics_test_EVAP.csv
   Evaluando variable: TMAX
      [OK] Guardado: /TMAX/metrics_test_TMAX.csv

## Validacion

In [ ]:
import pandas as pd
import numpy as np
import os
import joblib
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')

directory = {
    25037: {
        'Clasificacion': 'Seco',
        'ruta_input_val': './XGBOOST_dataset_splits/Seco/NR_25037/val/val_data.csv',
        'ruta_salida': './data_analysis/Performance_metrics/Seco/NR_25037/Validation_Imputation/',
        'ruta_modelo_imputacion': './XGBOOST_trained_models/Seco/NR_25037/imputer_model.joblib'
    },
    25033: {
        'Clasificacion': 'Templado',
        'ruta_input_val': './XGBOOST_dataset_splits/Templado/NR_25033/val/val_data.csv',
        'ruta_salida': './data_analysis/Performance_metrics/Templado/NR_25033/Validation_Imputation/',
        'ruta_modelo_imputacion': './XGBOOST_trained_models/Templado/NR_25033/imputer_model.joblib'
    },
    25046: {
        'Clasificacion': 'Tropical',
        'ruta_input_val': './XGBOOST_dataset_splits/Tropical/NR_25046/val/val_data.csv',
        'ruta_salida': './data_analysis/Performance_metrics/Tropical/NR_25046/Validation_Imputation/',
        'ruta_modelo_imputacion': './XGBOOST_trained_models/Tropical/NR_25046/imputer_model.joblib'
    },
}

columnas_objetivo = ['PRECIP', 'EVAP', 'TMAX', 'TMIN']
horizontes = [0, 3, 7, 31, 180, 365, 730, float('inf')]
etiquetas_horizontes = ['1d', '3d', '7d', '31d', '180d', '365d', '730d', 'full']


for estacion, info in directory.items():
    val_path = info['ruta_input_val']
    modelo_imputacion_path = info['ruta_modelo_imputacion']
    ruta_base_salida = info['ruta_salida']
    
    print(f"\nProcesando Estación: {estacion}")

    if not os.path.exists(val_path):
        print(f"   [SKIP] No existe archivo: {val_path}")
        continue

    df_old = pd.read_csv(val_path, index_col=0, parse_dates=True).asfreq('D')
    
    try:
        imputer = joblib.load(modelo_imputacion_path)
    except Exception as e:
        print(f"   [ERROR] Imputer no cargado: {e}")
        continue
    
    #Contexto Temporal
    df_ctx = df_old.copy()
    df_ctx['dayofyear'] = df_ctx.index.dayofyear
    df_ctx['month'] = df_ctx.index.month
    df_ctx['sin_dayofyear'] = np.sin(2 * np.pi * df_ctx.index.dayofyear / 365.25)
    df_ctx['cos_dayofyear'] = np.cos(2 * np.pi * df_ctx.index.dayofyear / 365.25)
    
    cols_mice = columnas_objetivo + ['dayofyear', 'month', 'sin_dayofyear', 'cos_dayofyear']

    try:
        matriz_nueva = imputer.transform(df_ctx[cols_mice])
        df_new = pd.DataFrame(matriz_nueva, columns=cols_mice, index=df_old.index)
    except Exception as e:
        print(f"   [ERROR] Falló imputación global: {e}")
        continue

    for objetivo in columnas_objetivo:
        if objetivo not in df_old.columns: continue
        
        print(f"   -> Generando reportes para: {objetivo}")
        

        dir_variable = os.path.join(ruta_base_salida, objetivo)
        os.makedirs(dir_variable, exist_ok=True)

        #Real vs Imputado
        df_comp = pd.DataFrame(index=df_old.index)
        df_comp['Original'] = df_old[objetivo]
        df_comp['Imputado'] = df_new[objetivo]
        df_comp['Fue_Rellenado'] = df_old[objetivo].isna()
        
        # Guardar en la carpeta de la variable
        df_comp.to_csv(os.path.join(dir_variable, 'comparativo_old_vs_new.csv'))
        
       # Log de Cambios
        huecos = df_comp[df_comp['Fue_Rellenado'] == True].copy()
        if not huecos.empty:
            huecos[['Original', 'Imputado']].to_csv(os.path.join(dir_variable, 'log_imputaciones_realizadas.csv'))
        
       # Validación Métricas
        indices_validos = df_ctx[df_ctx[objetivo].notna()].index
        
        if len(indices_validos) > 0:
            df_mask = df_ctx.copy()
            ground_truth = df_mask.loc[indices_validos, objetivo].copy()
            df_mask.loc[indices_validos, objetivo] = np.nan 
            
            try:
                matriz_val = imputer.transform(df_mask[cols_mice])
                df_val_res = pd.DataFrame(matriz_val, columns=cols_mice, index=df_mask.index)
                y_pred = df_val_res.loc[indices_validos, objetivo]
                
                df_res = pd.DataFrame({'Real': ground_truth, 'Pred': y_pred})
                start = df_res.index.min()
                df_res['Dias'] = (df_res.index - start).days + 1
                df_res['Rango'] = pd.cut(df_res['Dias'], bins=[-1] + horizontes, labels=etiquetas_horizontes)
                
                metrics_list = []
                for label in etiquetas_horizontes:
                    sub = df_res[df_res['Rango'] == label]
                    count = len(sub)
                    if count > 0:
                        mse = np.mean((sub['Real'] - sub['Pred'])**2)
                        rmse = np.sqrt(mse)
                        mae = np.mean(np.abs(sub['Real'] - sub['Pred']))
                        
                        mask_nz = sub['Real'] != 0
                        if mask_nz.sum() > 0:
                            mape = np.mean(np.abs((sub.loc[mask_nz, 'Real'] - sub.loc[mask_nz, 'Pred']) / sub.loc[mask_nz, 'Real']))
                        else:
                            mape = 0.0
                    else:
                        mse, rmse, mae, mape = 0,0,0,0
                    
                    metrics_list.append({
                        'Horizonte': label, 'MSE': round(mse,4), 'RMSE': round(rmse,4), 
                        'MAE': round(mae,4), 'MAPE': round(mape,4), 'Count': count
                    })
                
                # Guardar Metricas en la carpeta de la variable
                df_m = pd.DataFrame(metrics_list)
                df_m.to_csv(os.path.join(dir_variable, 'metricas_validacion.csv'), index=False)
                
                # Imagen de Tabla
                fig, ax = plt.subplots(figsize=(10, 3))
                ax.axis('off')
                tbl = ax.table(cellText=df_m.values, colLabels=df_m.columns, loc='center', cellLoc='center')
                tbl.scale(1, 1.4)
                plt.title(f'Validación Tabla: {objetivo}')
                plt.savefig(os.path.join(dir_variable, 'tabla_metricas.png'), bbox_inches='tight')
                plt.close()

                limit_plot = 150
                
                plt.figure(figsize=(12, 5))
                # Graficamos datos reales
                plt.plot(df_res['Real'].iloc[:limit_plot].values, label='Real (Validación)', color='blue', alpha=0.4)
                # Graficamos datos imputados
                plt.plot(df_res['Pred'].iloc[:limit_plot].values, label='Reconstrucción (XGBoost)', color='orange', linestyle='--', alpha=0.8)
                
                plt.title(f'Validación Reconstrucción: {objetivo} - Estación {estacion}')
                plt.legend()
                plt.grid(True, alpha=0.3)
                
                # Guardar gráfica en la carpeta de la variable
                plt.savefig(os.path.join(dir_variable, f'plot_validacion_{objetivo}.png'))
                plt.close()
                # ---------------------------------------------------------

            except Exception as e:
                print(f"      [ERROR Masking] {e}")

print("\n--- PROCESO TERMINADO ---")


Procesando Estación: 25037
   -> Generando reportes para: PRECIP
   -> Generando reportes para: EVAP
   -> Generando reportes para: TMAX
   -> Generando reportes para: TMIN

Procesando Estación: 25033
   -> Generando reportes para: PRECIP
   -> Generando reportes para: EVAP
   -> Generando reportes para: TMAX
   -> Generando reportes para: TMIN

Procesando Estación: 25046
   -> Generando reportes para: PRECIP
   -> Generando reportes para: EVAP
   -> Generando reportes para: TMAX
   -> Generando reportes para: TMIN

--- PROCESO TERMINADO ---
